In [1]:
import string
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.converter import XMLConverter, HTMLConverter, TextConverter
from pdfminer.image import ImageWriter
from pdfminer.layout import LAParams

import io
import os

import nltk
from nltk import word_tokenize, sent_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
import numpy as np
import pandas as pd

import json
from pathlib import *

import time

from random import randint

from nltk.corpus import wordnet

import operator

from nltk.chunk import tree2conlltags
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

import re

import numpy as np

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Pooja Ajit\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to C:\Users\Pooja
[nltk_data]     Ajit\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to C:\Users\Pooja
[nltk_data]     Ajit\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [19]:
def pdfRender():
    global documentSet
    global mydoc
    mydoc ={}
    pdf_files =[]
    allLines =[]
    FILE_PATH = Path(r'Final_LDA+KMeans')
    #FILE_PATH = Path('E:/MasterThesis/FinalPapers')
    pdf_files = list(FILE_PATH.glob('*.pdf'))
    #An Array which stores the full text of each document
    documentSet = pdfparser(pdf_files)
    mydoc = dict(zip(pdf_files,documentSet))
    #print(len(documentSet))
    return documentSet

In [20]:
def processData(rawContents):    
    cleaned = tokenizeContent(rawContents)    
    cleaned1 = removeStopWordsFromTokenized(cleaned)    
    cleaned2 = performPorterStemmingOnContents(cleaned1)    
    cleaned3 = removePunctuationFromTokenized(cleaned2)    
    cleaned4 = convertItemsToLower(cleaned3)    
    return cleaned4    
        
def tokenizeContent(contentsRaw):    
    tokenized = nltk.tokenize.sent_tokenize(contentsRaw)    
    return tokenized    
    
def removeStopWordsFromTokenized(contentsTokenized):    
    stop_word_set = set(nltk.corpus.stopwords.words("english"))    
    filteredContents = [word for word in contentsTokenized if word not in stop_word_set]    
    return filteredContents    
    
def performPorterStemmingOnContents(contentsTokenized):    
    porterStemmer = nltk.stem.PorterStemmer()    
    filteredContents = [porterStemmer.stem(word) for word in contentsTokenized]    
    return filteredContents    
    
def removePunctuationFromTokenized(contentsTokenized):    
    excludePuncuation = set(string.punctuation)    
    
    # manually add additional punctuation to remove    
    doubleSingleQuote = '\'\''    
    doubleDash = '--'    
    doubleTick = '``'    
    
    excludePuncuation.add(doubleSingleQuote)    
    excludePuncuation.add(doubleDash)    
    excludePuncuation.add(doubleTick)    
    
    filteredContents = [word for word in contentsTokenized if word not in excludePuncuation]    
    return filteredContents    
    
def convertItemsToLower(contentsRaw):    
    filteredContents = [term.lower() for term in contentsRaw]    
    return filteredContents


In [22]:
global fileName
def pdfparser(pdffileS):
    global finalDocumentSet
    finalDocumentSet = []
    global pdfEx
    pdfEx = []
    global files
    global fullText
    files = []
    fileName = []
    for pdffile in pdffileS:
        #full= fullText
        # Create a example words list(Please add all the related keywords needed)
        words_list = ["Introduction", "INTRODUCTION", "Background", "BACKGROUND", "Conclusion", "Conclusions",
                      "CONCLUSION", "Acknowledgements"]
        #print(words_list)
        pdfName = os.path.basename(pdffile)
        files.append(pdfName)
        fileName.append(pdfName)
        with open(pdffile, mode='rb') as f:
            fullText = np.array([])
            print(pdfName)
            #files.append(pdfName)
            #documents = fullText
            #words_list = []
            #print(words_list)
            #fp = open(data, 'rb')
            rsrcmgr = PDFResourceManager()
            retstr = io.StringIO()
            codec = 'utf-8'
            laparams = LAParams()
            data =[]
            details_page = []
            abstract = []
            device = TextConverter(rsrcmgr, retstr, laparams=laparams)
            # Create a PDF interpreter object.
            interpreter = PDFPageInterpreter(rsrcmgr, device)
            # Process each page contained in the document.
            count = 0
            for page in PDFPage.get_pages(f):
                interpreter.process_page(page)
                data = retstr.getvalue()
                details_page.append(data)

            #print("There are", len(words_list), "in the words list")
            stri = " "
            details = stri.join(details_page)
            words = details.split()
            place = []
            dummy_check = []
            removed_words = []

            print(words_list)
            for c, a in enumerate(words):
                for b in words_list:
                    if b == a and b not in dummy_check:
                        print(b, a)
                        place.append(details.find("{}".format(b)))
                        dummy_check.append(b)
                    #  place.append(words.index(a))
                    elif b not in words:
                        print(b)
                        removed_words.append(b)
                        words_list.remove(b)
                        print("The word", b, "was not found in the pdf file")

            #print(list(zip(words_list, place)))
            final_array = list(zip(words_list, place))
            #final_array.sort()
            final_array.sort(key=operator.itemgetter(1))
            # print("Sorting the final array")
            #print(final_array)

            # print("Extracting the relevant texts from pdf")
            # print(" ")
            print(final_array)
            if len(final_array) > 1:
                listint = final_array[0]
                list2int = final_array[1]
                counter = 0

                for each in (final_array):
                    if counter < len(final_array) - 2:
                        new = (details.split(listint[0])[1].split(list2int[0])[0])
                        #new = sent_tokenize(new)
                        #print(listint[0], ":", [' '.join(new)])
                        #print(" ")
                        #print(new)
                        #documents.append(new)
                        fullText = np.append(fullText, new)
                        counter = counter + 1
                        listint = final_array[0 + counter]
                        list2int = final_array[1 + counter]

                    elif counter < len(final_array) - 1:
                        new = (details.split(final_array[counter][0])[1].split(final_array[counter + 1][0])[0])
                        #new = sent_tokenize(new)
                        #documents.append(new)
                        fullText = np.append(fullText, new)
                        #print(final_array[counter][0], ":", [' '.join(new)])
                        #print(" ")
                        counter = counter + 1

                    else:
                        new = (details.split(final_array[counter][0])[1])
                        #new = sent_tokenize(new)
                        #documents.append(new)
                        fullText = np.append(fullText, new)
                        #print(final_array[counter][0], ":", [' '.join(new)])
                        #print(" ")
            else:
                new = (details.split(final_array[0][0])[1])
                # new = sent_tokenize(new)
                #documents.append(new)
                fullText = np.append(fullText, new)
                # print(final_array[counter][0], ":", [' '.join(new)])
                # print(" ")
                
        #finalDocumentSet = {pdfName : fullText}
        
        data=finalDocumentSet.append(fullText)
        myName=pdfEx.append(pdfName)
        #print("Testing==",finalDocumentSet)
        #data = finalDocumentSet.get(pdfName)
        #finalDocumentSet = finalDocumentSet
        data = str(data)
        
        data = processData(data)
        #data = data.replace(r'\\n', "")
        data = [i.replace('\\n', "") for i in data]
        data = [i.replace('\\x0', "") for i in data]
        words_list = words_list + removed_words
        print("Updated words list:")
        print(words_list)

    #print(len(finalDocumentSet))
    
    #mydoc = dict(zip(myName,data))
    return data


In [24]:
pdfRender()

10.1002@asi.23250.pdf
['Introduction', 'INTRODUCTION', 'Background', 'BACKGROUND', 'Conclusion', 'Conclusions', 'CONCLUSION', 'Acknowledgements']
INTRODUCTION
The word INTRODUCTION was not found in the pdf file
BACKGROUND
The word BACKGROUND was not found in the pdf file
Conclusions
The word Conclusions was not found in the pdf file
Acknowledgements
The word Acknowledgements was not found in the pdf file
Background
The word Background was not found in the pdf file
CONCLUSION
The word CONCLUSION was not found in the pdf file
Introduction Introduction
Conclusion Conclusion
[('Introduction', 1649), ('Conclusion', 10356)]
Updated words list:
['Introduction', 'Conclusion', 'INTRODUCTION', 'BACKGROUND', 'Conclusions', 'Acknowledgements', 'Background', 'CONCLUSION']
A-systematic-literature-review-of-actionable-alert-identification-techniques-for-automated-static-code-analysis2011Information-and-Software-Technology.pdf
['Introduction', 'INTRODUCTION', 'Background', 'BACKGROUND', 'Conclusion', 

Introduction Introduction
Background Background
Conclusion Conclusion
[('Introduction', 2844), ('Background', 10859), ('Conclusion', 683784)]
Updated words list:
['Introduction', 'Background', 'Conclusion', 'INTRODUCTION', 'BACKGROUND', 'Conclusions', 'Acknowledgements', 'CONCLUSION']
Identification-and-prioritization-of-SLR-search-tool-requirements-an-SLR-and-a-survey2019Empirical-Software-Engineering.pdf
['Introduction', 'INTRODUCTION', 'Background', 'BACKGROUND', 'Conclusion', 'Conclusions', 'CONCLUSION', 'Acknowledgements']
INTRODUCTION
The word INTRODUCTION was not found in the pdf file
BACKGROUND
The word BACKGROUND was not found in the pdf file
CONCLUSION
The word CONCLUSION was not found in the pdf file
Background
The word Background was not found in the pdf file
Acknowledgements
The word Acknowledgements was not found in the pdf file
Conclusions Conclusions
Introduction Introduction
Conclusion Conclusion
[('Introduction', 1756), ('Conclusions', 1756), ('Conclusion', 4512)]
Upd

['Introduction', 'INTRODUCTION', 'Background', 'BACKGROUND', 'Conclusion', 'Conclusions', 'CONCLUSION', 'Acknowledgements']
Introduction
The word Introduction was not found in the pdf file
Conclusion
The word Conclusion was not found in the pdf file
Acknowledgements
The word Acknowledgements was not found in the pdf file
INTRODUCTION
The word INTRODUCTION was not found in the pdf file
Conclusions
The word Conclusions was not found in the pdf file
Background Background
BACKGROUND BACKGROUND
CONCLUSION CONCLUSION
[('Background', 654), ('BACKGROUND', 1866), ('CONCLUSION', 89245)]
Updated words list:
['Background', 'BACKGROUND', 'CONCLUSION', 'Introduction', 'Conclusion', 'Acknowledgements', 'INTRODUCTION', 'Conclusions']
Online-knowledge-sharing-mechanisms-a-systematic-review-of-the-state-of-the-art-literature-and-recommendations-for-future-research2016Information-Systems-Frontiers.pdf
['Introduction', 'INTRODUCTION', 'Background', 'BACKGROUND', 'Conclusion', 'Conclusions', 'CONCLUSION', 

Conclusions Conclusions
[('Introduction', 2085), ('Conclusions', 427889)]
Updated words list:
['Introduction', 'Conclusions', 'INTRODUCTION', 'BACKGROUND', 'CONCLUSION', 'Background', 'Acknowledgements', 'Conclusion']
shemilt2013.pdf
['Introduction', 'INTRODUCTION', 'Background', 'BACKGROUND', 'Conclusion', 'Conclusions', 'CONCLUSION', 'Acknowledgements']
INTRODUCTION
The word INTRODUCTION was not found in the pdf file
BACKGROUND
The word BACKGROUND was not found in the pdf file
Conclusions
The word Conclusions was not found in the pdf file
Background
The word Background was not found in the pdf file
CONCLUSION
The word CONCLUSION was not found in the pdf file
Conclusion
The word Conclusion was not found in the pdf file
Introduction Introduction
Acknowledgements Acknowledgements
[('Introduction', 2120), ('Acknowledgements', 703866)]
Updated words list:
['Introduction', 'Acknowledgements', 'INTRODUCTION', 'BACKGROUND', 'Conclusions', 'Background', 'CONCLUSION', 'Conclusion']
Supporting-

['none']

In [25]:
len(finalDocumentSet)

39

In [26]:
len(files)


39

In [28]:
files[0]

'10.1002@asi.23250.pdf'

In [29]:
#create a dictionary to save the pdf name and the objectives for QA questions
final_dataset_QA = {'fileName': [], 'content': [] }
for a,b in zip(files, finalDocumentSet):    
    final_dataset_QA['fileName'].append(a)
    final_dataset_QA['content'].append(b)

In [30]:
#export the results to .xslx
from pandas import DataFrame

df = DataFrame(final_dataset_QA, columns= ['fileName', 'content'])
export_excel = df.to_excel (r'E:\MasterThesis\FinalPapers\excel_QA\DataSet_QA_31.xlsx', index = None, header=True) #Don't forget to add '.xlsx' at the end of the path



In [33]:
#extract introduction, methods, conclusion portions of the paper
extractedContent = []
global keyword
def extractIntroduction():
    count = 0
    for i in range(len(finalDocumentSet)):
        doc = str(finalDocumentSet[i])
        intro=re.search("|".join([r'Background(.*?)Methods', r'2. Study objective(.*?)3. Methods', r'Introduction(.*?)Methods', r'Purpose(.*?)Methods', r'Introduction(.*?)Overview', r'Introduction(.*?)Method', r'Background(.*?)Main Text', r'Introduction(.*?)Methodology', r'Introduction(.*?) Related work', r'Context(.*?)Methods', r'Introduction(.*?)Methodology', r'Objectives(.*?)Methods', r'Introduction(.*?)Study Objective', r'Objective(.*?)Method', r'Introduction(.*?)Background' ]), str(doc)).group()           
        print("Introduction is extracted successfully")
        extractedContent.append(intro) 
        return extractedContent 

In [35]:
other_content = []
for i in range(len(finalDocumentSet)):
    content = str(finalDocumentSet[i])
    #Text pre-processing
    content = content.lower()
    content = re.sub(r'\d+', '', content)
    #data = re.sub(r'[a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[a-z]+','', data)
    content = content.replace("\n", "")
    #data = data.strip()
    content = sent_tokenize(content)
    other_content.append(content)

In [37]:
len(other_content)

39

In [38]:
pip install rank-bm25

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [39]:
from rank_bm25 import BM25Okapi

In [40]:
def create_tokens_questions(question):
    tokenized_question = question.split()
    create_chunks = nltk.ne_chunk(nltk.pos_tag(tokenized_question))
    assign_pos_tags = tree2conlltags(create_chunks)
    return assign_pos_tags

In [41]:
def extract_pos(question):
    noun_add = []
    assign_pos_tags = create_tokens_questions(question)
    print(assign_pos_tags)
    for i in range(0, len(assign_pos_tags) - 1):
        if (assign_pos_tags[i][1] == 'NN' and assign_pos_tags[i + 1][1] == 'NN'):
            extract_nouns = assign_pos_tags[i] + assign_pos_tags[i + 1]
            noun_add.append(extract_nouns)
        elif assign_pos_tags[i][1] == 'NNP':
            extract_nouns = assign_pos_tags[i]
            print("The word", extract_nouns[0], "is extracted and added to the query list")
            noun_add.append(extract_nouns[0])
        elif assign_pos_tags[i][1] == 'NN':
            extract_nouns = assign_pos_tags[i]
            print("The word", extract_nouns[0], "is extracted and added to the query list")
            noun_add.append(extract_nouns[0])
    return noun_add

In [42]:
introFinalData = []
global keyword
def extractIntroduction(inputDoc):
    for j in range(len(inputDoc)):
        question = "what is the aim and objective of this paper?"
        tokenized_corpus = [doc.split(" ") for doc in inputDoc[j]]
        bm25 = BM25Okapi(tokenized_corpus)
        noun_add = extract_pos(question)
        #print(noun_add)
        get_final_results = bm25.get_top_n(noun_add, inputDoc[j], n=5)
        introFinalData.append(get_final_results)
        #print(get_final_results)
    return introFinalData


In [43]:
otherContent = []
global keyword
def extractMethods(inputDoc):
    for j in range(len(inputDoc)):
        question = "what is the method used in this paper?"
        tokenized_corpus = [doc.split(" ") for doc in inputDoc[j]]
        bm25 = BM25Okapi(tokenized_corpus)
        noun_add = extract_pos(question)
        #print(noun_add)
        get_final_results = bm25.get_top_n(noun_add, inputDoc[j], n=10)
        otherContent.append(get_final_results)
        #print(get_final_results)
    return otherContent

In [58]:
Conclu = []
global keyword
def extractConclusion(inputDoc):
    for j in range(len(inputDoc)):
        question = "what is the conclusion and discussion and justification of this paper?"
        tokenized_corpus = [doc.split(" ") for doc in inputDoc[j]]
        bm25 = BM25Okapi(tokenized_corpus)
        noun_add = extract_pos(question)
        #print(noun_add)
        get_final_results = bm25.get_top_n(noun_add, inputDoc[j], n=10)
        Conclu.append(get_final_results)
        #print(get_final_results)
    return Conclu

In [45]:
#To fetch the conclusion for each paper
QA1 = extractIntroduction(other_content)

[('what', 'WP', 'O'), ('is', 'VBZ', 'O'), ('the', 'DT', 'O'), ('aim', 'NN', 'O'), ('and', 'CC', 'O'), ('objective', 'NN', 'O'), ('of', 'IN', 'O'), ('this', 'DT', 'O'), ('paper?', 'NN', 'O')]
The word aim is extracted and added to the query list
The word objective is extracted and added to the query list
[('what', 'WP', 'O'), ('is', 'VBZ', 'O'), ('the', 'DT', 'O'), ('aim', 'NN', 'O'), ('and', 'CC', 'O'), ('objective', 'NN', 'O'), ('of', 'IN', 'O'), ('this', 'DT', 'O'), ('paper?', 'NN', 'O')]
The word aim is extracted and added to the query list
The word objective is extracted and added to the query list
[('what', 'WP', 'O'), ('is', 'VBZ', 'O'), ('the', 'DT', 'O'), ('aim', 'NN', 'O'), ('and', 'CC', 'O'), ('objective', 'NN', 'O'), ('of', 'IN', 'O'), ('this', 'DT', 'O'), ('paper?', 'NN', 'O')]
The word aim is extracted and added to the query list
The word objective is extracted and added to the query list
[('what', 'WP', 'O'), ('is', 'VBZ', 'O'), ('the', 'DT', 'O'), ('aim', 'NN', 'O'), ('a

[('what', 'WP', 'O'), ('is', 'VBZ', 'O'), ('the', 'DT', 'O'), ('aim', 'NN', 'O'), ('and', 'CC', 'O'), ('objective', 'NN', 'O'), ('of', 'IN', 'O'), ('this', 'DT', 'O'), ('paper?', 'NN', 'O')]
The word aim is extracted and added to the query list
The word objective is extracted and added to the query list
[('what', 'WP', 'O'), ('is', 'VBZ', 'O'), ('the', 'DT', 'O'), ('aim', 'NN', 'O'), ('and', 'CC', 'O'), ('objective', 'NN', 'O'), ('of', 'IN', 'O'), ('this', 'DT', 'O'), ('paper?', 'NN', 'O')]
The word aim is extracted and added to the query list
The word objective is extracted and added to the query list
[('what', 'WP', 'O'), ('is', 'VBZ', 'O'), ('the', 'DT', 'O'), ('aim', 'NN', 'O'), ('and', 'CC', 'O'), ('objective', 'NN', 'O'), ('of', 'IN', 'O'), ('this', 'DT', 'O'), ('paper?', 'NN', 'O')]
The word aim is extracted and added to the query list
The word objective is extracted and added to the query list
[('what', 'WP', 'O'), ('is', 'VBZ', 'O'), ('the', 'DT', 'O'), ('aim', 'NN', 'O'), ('a

In [46]:
len(QA1)

39

In [47]:
QA1

[['warner’s () differentiation between\\nthe computer science traditions and an older library-\\noriented tradition seems important; the former aim to\\ntransform queries automatically into (ranked) sets of\\nrelevant documents, whereas the latter aims to increase\\nthe “selection power” of users.',
  'warner’s () differentiation between\\nthe computer science traditions and an older library-\\noriented tradition seems important; the former aim to\\ntransform queries automatically into (ranked) sets of\\nrelevant documents, whereas the latter aims to increase\\nthe “selection power” of users.',
  "this section also contains a subsection that pres-\\nents empirical studies on searching for systematic reviews.\\nin the ']",
  'the term information retrieval\\nwas introduced by mooers (–), who deﬁned it\\nthus:\\n\\ninformation retrieval is the name for the process or method\\nwhereby a prospective user of information is able to convert his\\nneed for information into an actual list of ci

In [48]:
#To fetch the methods for each paper
QA2 = extractMethods(other_content)

[('what', 'WP', 'O'), ('is', 'VBZ', 'O'), ('the', 'DT', 'O'), ('method', 'NN', 'O'), ('used', 'VBN', 'O'), ('in', 'IN', 'O'), ('this', 'DT', 'O'), ('paper?', 'NN', 'O')]
The word method is extracted and added to the query list
[('what', 'WP', 'O'), ('is', 'VBZ', 'O'), ('the', 'DT', 'O'), ('method', 'NN', 'O'), ('used', 'VBN', 'O'), ('in', 'IN', 'O'), ('this', 'DT', 'O'), ('paper?', 'NN', 'O')]
The word method is extracted and added to the query list
[('what', 'WP', 'O'), ('is', 'VBZ', 'O'), ('the', 'DT', 'O'), ('method', 'NN', 'O'), ('used', 'VBN', 'O'), ('in', 'IN', 'O'), ('this', 'DT', 'O'), ('paper?', 'NN', 'O')]
The word method is extracted and added to the query list
[('what', 'WP', 'O'), ('is', 'VBZ', 'O'), ('the', 'DT', 'O'), ('method', 'NN', 'O'), ('used', 'VBN', 'O'), ('in', 'IN', 'O'), ('this', 'DT', 'O'), ('paper?', 'NN', 'O')]
The word method is extracted and added to the query list
[('what', 'WP', 'O'), ('is', 'VBZ', 'O'), ('the', 'DT', 'O'), ('method', 'NN', 'O'), ('used'

In [49]:
QA2

[["this section also contains a subsection that pres-\\nents empirical studies on searching for systematic reviews.\\nin the ']",
  'the term information retrieval\\nwas introduced by mooers (–), who deﬁned it\\nthus:\\n\\ninformation retrieval is the name for the process or method\\nwhereby a prospective user of information is able to convert his\\nneed for information into an actual list of citations to documents\\nin storage containing information useful to him.',
  'the paper examines this claim and argues for\\nthe continued value of boolean systems, and suggests\\ntwo further considerations: (a) the important role of\\nhuman expertise in searching (expert searchers and\\n“information literate” users) and (b) the role of library\\nand information science and knowledge organization\\n(ko) in the design and use of classical databases.',
  'an\\nunderlying issue is the kind of retrieval system for which\\none should aim.',
  'warner’s () differentiation between\\nthe computer science

In [50]:
QA2[1]

['\\noverview of systematic literature review method .',
 '\\noverview of systematic literature review method .',
 '\\noverview of systematic literature review method .',
 "\\n']",
 '.',
 '.',
 '.',
 '.',
 '.',
 '.']

In [59]:
#To fetch the conclusion for each paper
QA3 = extractConclusion(other_content)

[('what', 'WP', 'O'), ('is', 'VBZ', 'O'), ('the', 'DT', 'O'), ('conclusion', 'NN', 'O'), ('and', 'CC', 'O'), ('discussion', 'NN', 'O'), ('and', 'CC', 'O'), ('justification', 'NN', 'O'), ('of', 'IN', 'O'), ('this', 'DT', 'O'), ('paper?', 'NN', 'O')]
The word conclusion is extracted and added to the query list
The word discussion is extracted and added to the query list
The word justification is extracted and added to the query list
[('what', 'WP', 'O'), ('is', 'VBZ', 'O'), ('the', 'DT', 'O'), ('conclusion', 'NN', 'O'), ('and', 'CC', 'O'), ('discussion', 'NN', 'O'), ('and', 'CC', 'O'), ('justification', 'NN', 'O'), ('of', 'IN', 'O'), ('this', 'DT', 'O'), ('paper?', 'NN', 'O')]
The word conclusion is extracted and added to the query list
The word discussion is extracted and added to the query list
The word justification is extracted and added to the query list
[('what', 'WP', 'O'), ('is', 'VBZ', 'O'), ('the', 'DT', 'O'), ('conclusion', 'NN', 'O'), ('and', 'CC', 'O'), ('discussion', 'NN', 

[('what', 'WP', 'O'), ('is', 'VBZ', 'O'), ('the', 'DT', 'O'), ('conclusion', 'NN', 'O'), ('and', 'CC', 'O'), ('discussion', 'NN', 'O'), ('and', 'CC', 'O'), ('justification', 'NN', 'O'), ('of', 'IN', 'O'), ('this', 'DT', 'O'), ('paper?', 'NN', 'O')]
The word conclusion is extracted and added to the query list
The word discussion is extracted and added to the query list
The word justification is extracted and added to the query list
[('what', 'WP', 'O'), ('is', 'VBZ', 'O'), ('the', 'DT', 'O'), ('conclusion', 'NN', 'O'), ('and', 'CC', 'O'), ('discussion', 'NN', 'O'), ('and', 'CC', 'O'), ('justification', 'NN', 'O'), ('of', 'IN', 'O'), ('this', 'DT', 'O'), ('paper?', 'NN', 'O')]
The word conclusion is extracted and added to the query list
The word discussion is extracted and added to the query list
The word justification is extracted and added to the query list
[('what', 'WP', 'O'), ('is', 'VBZ', 'O'), ('the', 'DT', 'O'), ('conclusion', 'NN', 'O'), ('and', 'CC', 'O'), ('discussion', 'NN', 

In [52]:
len(QA1)

39

In [53]:
len(QA2)

39

In [60]:
len(QA3)

39

In [61]:
global data_cleaned
def clean(data):
    data_cleaned=[]
    for line in data:
        line = line.lower()
        line = re.sub(r'\d+', '', line)
        #line = re.sub(r'[a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[a-z]+','', line)
        #line = line.translate(str.maketrans('','', string.punctuation))
        #line = line.strip()
        line = line.replace("\n", "")
        line = line.replace("\\n", "")
        #line = sent_tokenize(line)
        
        data_cleaned.append(line)
    return data_cleaned
    

In [62]:
QA1_Data = []
for k in range(len(QA1)):
    CleanedData = (QA1[k])
    CleanedData = clean(CleanedData)
    QA1_Data.append(CleanedData)


In [63]:
QA2_Data = []
for l in range(len(QA2)):
    CleanedData = (QA2[l])
    CleanedData = clean(CleanedData)
    QA2_Data.append(CleanedData)

In [64]:
QA3_Data = []
for m in range(len(QA3)):
    CleanedData = (QA1[m])
    CleanedData = clean(CleanedData)
    QA3_Data.append(CleanedData)

In [65]:
#create a dictionary to save the pdf name and the respective answers for the QA1
#Is the aim and the objective stated clearly?
QA1_Ans = {'PDFName': [], 'QA1_ans': [] }
for c,d in zip(files, QA1_Data):    
    QA1_Ans['PDFName'].append(c)
    QA1_Ans['QA1_ans'].append(d)


In [66]:
#create a dictionary to save the pdf name and the respective answers for the QA2
#This can answer two questions partially 
#Adequate description of the sample used and the methods for identifying and recruiting the sample
#Adequate description of methods used to analyse the data (partially)
QA2_Ans = {'PDFName': [], 'QA2_ans': [] }
for e,f in zip(files, QA2_Data):    
    QA2_Ans['PDFName'].append(e)
    QA2_Ans['QA2_ans'].append(f)

In [67]:
#create a dictionary to save the pdf name and the respective answers for the QA3
#Does the study provide clearly stated findings with credible results and justified conclusions
QA3_Ans = {'PDFName': [], 'QA3_ans': [] }
for g,h in zip(files, QA3_Data):    
    QA3_Ans['PDFName'].append(g)
    QA3_Ans['QA3_ans'].append(h)


In [68]:
#export the answers to the QA1,QA2,QA3 to .xslx

#the final results are exported to the excel to let the user read it and score/assess the paper accordingly
from pandas import DataFrame

df1 = DataFrame(QA1_Ans, columns= ['PDFName', 'QA1_ans'])
export_excel = df1.to_excel (r'E:\MasterThesis\FinalPapers\excel_QA\QA1_Answer.xlsx', index = None, header=True) #Don't forget to add '.xlsx' at the end of the path

df2 = DataFrame(QA2_Ans, columns= ['PDFName', 'QA2_ans'])
export_excel = df2.to_excel (r'E:\MasterThesis\FinalPapers\excel_QA\QA2_Answer.xlsx', index = None, header=True) #Don't forget to add '.xlsx' at the end of the path

df3 = DataFrame(QA3_Ans, columns= ['PDFName', 'QA3_ans'])
export_excel = df3.to_excel (r'E:\MasterThesis\FinalPapers\excel_QA\QA3_Answer.xlsx', index = None, header=True) #Don't forget to add '.xlsx' at the end of the path
